In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split


In [5]:
train = pd.read_csv('data/challenge3_train.csv', index_col='id')
test = pd.read_csv('data/challenge3_test.csv', index_col='id')
full_data = pd.concat([train, test]).drop(columns=['target'])

#X_train, X_test, y_train, y_test = train_test_split(train.drop(columns='target'), train['target'])

In [33]:
features_bin = ['f0', 'f26'] # binary
features_nom_low = ['f5'] # nominal low cardinality (<=3)
features_nom_high = ['f12', 'f28'] # >= 26
features_ord_low = ['f1', 'f2', 'f3', 'f6', 'f9', 'f11', 'f13', 'f15', 'f17', 'f18', 'f19', 'f20', 'f21', 'f23', 'f24', 'f25', 'f27', 'f29']
features_ord_num = ['f1', 'f2', 'f3', 'f6', 'f9', 'f11', 'f13', 'f17', 'f18', 'f19', 'f21', 'f23', 'f24', 'f25', 'f27', 'f29']
features_ord_high = ['f4', 'f10', 'f14']
features_real = ['f7', 'f8', 'f16', 'f22', 'f30'] # real numbers.

features_ord_alph = ['f15', 'f20']

## First clean data for noise etc.

In [6]:
# Fix 0 value noise, change to most common value in column
impute_0_columns = ['f3', 'f18', 'f21']

for column in impute_0_columns:
    train.loc[train[column] == 0, column] = train[column].mode() # mode() finds most common value
    test.loc[test[column] == 0, column] = test[column].mode()

In [ ]:
# cyclic -> sin cos (month). f11
# ordinal with numbers: no need to do anything

# Feature enfactorizeing

In [29]:
# Alphabetic -> numbers (a-1, b-2 etc). f15, f20
for col in features_ord_alph:
    ord_order_dict = {i : j for j, i in enumerate(sorted(list(set(list(train[col].dropna().unique()) + list(test[col].dropna().unique())))))}
    train[f'{col}_en'] = train[col].fillna('NULL').map(ord_order_dict)
    test[f'{col}_en'] = test[col].fillna('NULL').map(ord_order_dict)

In [46]:
# One hot encoding
full_data = pd.get_dummies(full_data, columns=features_nom_low+features_nom_high)

In [ ]:
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder


# another difficult column is f28 because of its high cardinality, if we were to use get_dummies, there would be an insane amount of columns
# in this column we will use target encoding, which is mean encoding with features to prevent bias
encoder = TargetEncoder()
challenge3_train['f28'] = encoder.fit_transform(challenge3_train['f12'], challenge3_train['target'])
